# SPAM CLASSIFIER

In [10]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import tarfile
import os

In [8]:
path = 'datasets\spam'
spam_file = r'20030228_spam.tar.bz2'
ham_file = r'20030228_easy_ham.tar.bz2'

In [11]:
tar_bz2_spam = tarfile.open(os.path.join(path, spam_file))
tar_bz2_spam.extractall(path=path)
tar_bz2_spam.close()

In [ ]:
tar_bz2_ham = tarfile.open(os.path.join(path, ham_file))
tar_bz2_ham.extractall(path=path)
tar_bz2_ham.close()